In [1]:
import pandas as pd
import glob

paths = glob.glob('./Football-Dataset/*/*')
pd_list = []

for path in paths:
    temp_csv = pd.read_csv(path)
    pd_list.append(temp_csv)

df = pd.concat(pd_list)

In [2]:
def clean_link(x):
    new_link = x.split('.com')[1]
    new_link_year = new_link.split('/')[-1][0:4]
    new_link_complete = '/'.join(new_link.split('/')[:-1]) + '/' + new_link_year
    return new_link_complete

In [3]:
df['New Link'] = df['Link'].apply(clean_link)

In [4]:
match_info_df = pd.read_csv('Match_Info.csv')

In [5]:
new_df = df.merge(match_info_df, left_on='New Link', right_on='Link').rename(columns={'Link_x': 'Link'})

In [6]:
new_df

,Home_Team,Away_Team,Result,Link,Season,Round,League,New Link,Link_y,Date_New,Referee,Home_Yellow,Home_Red,Away_Yellow,Away_Red
0,Chelsea,Hull City,2-1,https://www.besoccer.com/match/chelsea-fc/hull...,2010,1,premier_league,/match/chelsea-fc/hull-city/2010,/match/chelsea-fc/hull-city/2010,"Saturday, 15 August 2009, 13:45",\r\nReferee: Alan Wiley\r\n,1.0,0.0,1.0,0.0
1,Aston Villa,Wigan Athletic,0-2,https://www.besoccer.com/match/aston-villa-fc/...,2010,1,premier_league,/match/aston-villa-fc/wigan-athletic/2010,/match/aston-villa-fc/wigan-athletic/2010,"Saturday, 15 August 2009, 16:00",\r\nReferee: Mark Clattenburg\r\n,2.0,0.0,2.0,0.0
2,Blackburn Rovers,Man. City,0-2,https://www.besoccer.com/match/blackburn-rover...,2010,1,premier_league,/match/blackburn-rovers-fc/manchester-city-fc/...,/match/blackburn-rovers-fc/manchester-city-fc/...,"Saturday, 15 August 2009, 16:00",\r\nReferee: Mike Dean\r\n,1.0,0.0,1.0,0.0
3,Bolton Wanderers,Sunderland,0-1,https://www.besoccer.com/match/bolton-wanderer...,2010,1,premier_league,/match/bolton-wanderers-fc/sunderland-afc/2010,/match/bolton-wanderers-fc/sunderland-afc/2010,"Saturday, 15 August 2009, 16:00",\r\nReferee: Andre Marriner\r\n,2.0,0.0,0.0,0.0
4,Portsmouth,Fulham,0-1,https://www.besoccer.com/match/portsmouth/fulh...,2010,1,premier_league,/match/portsmouth/fulham/2010,/match/portsmouth/fulham/2010,"Saturday, 15 August 2009, 16:00",\r\nReferee: Martin Atkinson\r\n,3.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142531,Huddersfield Town,Swindon Town,0-0,https://www.besoccer.com/match/huddersfield-to...,1997,46,championship,/match/huddersfield-town-fc/swindon-town/1997,/match/huddersfield-town-fc/swindon-town/1997,"Sunday, 04 May 1997, 00:00",\r\n,0.0,0.0,3.0,0.0
142532,Grimsby Town,Southend United,4-0,https://www.besoccer.com/match/grimsby-town/so...,1997,46,championship,/match/grimsby-town/southend-united/1997,/match/grimsby-town/southend-united/1997,"Sunday, 04 May 1997, 00:00",\r\n,2.0,0.0,1.0,0.0
142533,Crystal Palace,Port Vale,1-1,https://www.besoccer.com/match/crystal-palace-...,1997,46,championship,/match/crystal-palace-fc/port-vale/1997,/match/crystal-palace-fc/port-vale/1997,"Sunday, 04 May 1997, 00:00",\r\n,2.0,0.0,0.0,0.0
142534,Charlton Athletic,Sheffield United,0-0,https://www.besoccer.com/match/charlton-athlet...,1997,46,championship,/match/charlton-athletic-fc/sheffield-united/1997,/match/charlton-athletic-fc/sheffield-united/1997,"Sunday, 04 May 1997, 00:00",\r\n,3.0,0.0,2.0,0.0


In [7]:
import pickle

elo_dict = pickle.load(open('elo_dict.pkl', 'rb'))

In [8]:
elo_df = pd.DataFrame.from_dict(elo_dict)

In [9]:
elo_df = elo_df.transpose().reset_index().rename(columns={'index': 'Link'})

In [10]:
final_df = elo_df.merge(new_df, on='Link').dropna(subset=['Elo_home']).drop_duplicates(subset=['Link'], keep='first')

In [11]:
final_df

,Link,Elo_home,Elo_away,Home_Team,Away_Team,Result,Season,Round,League,New Link,Link_y,Date_New,Referee,Home_Yellow,Home_Red,Away_Yellow,Away_Red
0,https://www.besoccer.com/match/saarbrucken/stu...,56.0,59.0,Saarbrücken,Stuttgarter Kickers,1-0,1990,1,2_liga,/match/saarbrucken/stuttgarter-kickers/1990,/match/saarbrucken/stuttgarter-kickers/1990,"Saturday, 29 July 1989, 15:00",\r\nReferee: Hans-Jürgen Weber\r\n,0.0,0.0,3.0,0.0
1,https://www.besoccer.com/match/sc-freiburg/unt...,53.0,55.0,SC Freiburg,Unterhaching,2-3,1990,1,2_liga,/match/sc-freiburg/unterhaching/1990,/match/sc-freiburg/unterhaching/1990,"Saturday, 29 July 1989, 15:00",\r\nReferee: Kurt Wittke\r\n,1.0,0.0,0.0,0.0
2,https://www.besoccer.com/match/vfl-osnabruck/m...,52.0,53.0,VfL Osnabrück,Meppen,1-1,1990,1,2_liga,/match/vfl-osnabruck/meppen/1990,/match/vfl-osnabruck/meppen/1990,"Saturday, 29 July 1989, 15:00",\r\nReferee: Werner Föckler\r\n,3.0,0.0,2.0,0.0
3,https://www.besoccer.com/match/rot-weiss-essen...,53.0,62.0,RW Essen,Schalke 04,0-0,1990,1,2_liga,/match/rot-weiss-essen/schalke-04/1990,/match/rot-weiss-essen/schalke-04/1990,"Saturday, 29 July 1989, 15:00",\r\nReferee: Heinz Werner\r\n,2.0,0.0,2.0,0.0
4,https://www.besoccer.com/match/alemannia-aache...,57.0,52.0,Alemannia Aachen,MSV Duisburg,1-2,1990,1,2_liga,/match/alemannia-aachen/msv-duisburg/1990,/match/alemannia-aachen/msv-duisburg/1990,"Saturday, 29 July 1989, 15:00",\r\nReferee: Hans-Peter Dellwing\r\n,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142530,https://www.besoccer.com/match/pisa-calcio/vic...,55.0,48.0,Pisa,Vicenza,2-2,2021,25,serie_b,/match/pisa-calcio/vicenza-calcio/2021,/match/pisa-calcio/vicenza-calcio/2021,"Saturday, 27 February 2021, 14:00",\r\nReferee: Dionisi F.\r\n,5.0,0.0,1.0,0.0
142531,https://www.besoccer.com/match/pescara-calcio/...,58.0,59.0,Pescara,Lecce,1-1,2021,25,serie_b,/match/pescara-calcio/lecce/2021,/match/pescara-calcio/lecce/2021,"Saturday, 27 February 2021, 14:00",\r\nReferee: Robilotta I.\r\n,1.0,0.0,1.0,0.0
142532,https://www.besoccer.com/match/spal-1907/reggi...,60.0,39.0,SPAL,Reggina,1-4,2021,25,serie_b,/match/spal-1907/reggina/2021,/match/spal-1907/reggina/2021,"Saturday, 27 February 2021, 14:00",\r\nReferee: Prontera A.\r\n,3.0,1.0,4.0,0.0
142533,https://www.besoccer.com/match/ac-monza-brianz...,51.0,58.0,AC Monza,Cittadella,0-0,2021,25,serie_b,/match/ac-monza-brianza-1912/as-cittadella/2021,/match/ac-monza-brianza-1912/as-cittadella/2021,"Saturday, 27 February 2021, 16:00",\r\nReferee: Ros R.\r\n,2.0,0.0,4.0,0.0


In [16]:
new_df = final_df[['League', 'Round', 'Season', 'Home_Team', 'Away_Team', 'Result', 'Elo_home', 'Elo_away']]

In [17]:
def get_home_result(x):
    home = x.split('-')[0]
    return home

def get_away_result(x):
    away = x.split('-')[1]
    return away

new_df['Home Score'] = new_df['Result'].apply(get_home_result)
new_df['Away Score'] = new_df['Result'].apply(get_away_result)

<ipython-input-17-812c56d23de5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Home Score'] = new_df['Result'].apply(get_home_result)
<ipython-input-17-812c56d23de5>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Away Score'] = new_df['Result'].apply(get_away_result)


In [18]:
for idx, match in new_df.iterrows():
    if match['Home Score'] > match['Away Score']:
        res = 1
    elif match['Home Score'] < match['Away Score']:
        res = 3
    else:
        res = 2
    new_df.loc[ idx, 'Result_Num'] = res

/home/ivanyingx/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/ivanyingx/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [35]:
new_df = new_df.rename(columns={'Result_Num': 'Outcome'})

In [46]:
mask = (new_df['League'] == 'premier_league') & (new_df['Season'] == 2021)

In [58]:
premier_league = new_df[mask]

In [59]:
premier_league

,League,Round,Season,Home_Team,Away_Team,Result,Elo_home,Elo_away,Home Score,Away Score,Outcome
79243,premier_league,1,2021,Fulham,Arsenal,0-3,69.0,90.0,0,3,3.0
79244,premier_league,1,2021,Crystal Palace,Southampton,1-0,74.0,79.0,1,0,1.0
79245,premier_league,1,2021,Liverpool,Leeds United,4-3,96.0,69.0,4,3,1.0
79246,premier_league,1,2021,West Ham,Newcastle,0-2,76.0,74.0,0,2,3.0
79247,premier_league,1,2021,West Bromwich Albion,Leicester,0-3,79.0,83.0,0,3,3.0
...,...,...,...,...,...,...,...,...,...,...,...
79495,premier_league,26,2021,Crystal Palace,Fulham,0-0,75.0,69.0,0,0,2.0
79496,premier_league,26,2021,Leicester,Arsenal,1-3,84.0,89.0,1,3,3.0
79497,premier_league,26,2021,Tottenham Hotspur,Burnley,4-0,89.0,73.0,4,0,1.0
79498,premier_league,26,2021,Chelsea,Man. Utd,0-0,90.0,91.0,0,0,2.0


In [60]:
premier_league = premier_league[['Round', 'Season', 'Home_Team', 'Away_Team', 'Home Score', 'Away Score']]
premier_league

,Round,Season,Home_Team,Away_Team,Home Score,Away Score
79243,1,2021,Fulham,Arsenal,0,3
79244,1,2021,Crystal Palace,Southampton,1,0
79245,1,2021,Liverpool,Leeds United,4,3
79246,1,2021,West Ham,Newcastle,0,2
79247,1,2021,West Bromwich Albion,Leicester,0,3
...,...,...,...,...,...,...
79495,26,2021,Crystal Palace,Fulham,0,0
79496,26,2021,Leicester,Arsenal,1,3
79497,26,2021,Tottenham Hotspur,Burnley,4,0
79498,26,2021,Chelsea,Man. Utd,0,0


In [66]:
premier_league['Total Goals So Far Home'] = 0
premier_league['Total Goals So Far Away'] = 0


<ipython-input-66-799ca2c7e087>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  premier_league['Total Goals So Far Home'] = 0
<ipython-input-66-799ca2c7e087>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  premier_league['Total Goals So Far Away'] = 0


In [87]:
aux_dict = {}
for round in premier_league['Round'].unique()[:-1]:
    new_round = premier_league[premier_league['Round'] == round]
    for i, row in new_round.iterrows():
        if row['Home_Team'] in aux_dict:
            aux_dict[row['Home_Team']] += int(row['Home Score'])
        else:
            aux_dict[row['Home_Team']] = int(row['Home Score'])

        if row['Away_Team'] in aux_dict:
            aux_dict[row['Away_Team']] += int(row['Away Score'])
        else:
            aux_dict[row['Away_Team']] = int(row['Away Score'])        
    for team in aux_dict.keys():
        idx_home = premier_league[(premier_league['Home_Team'] == team) & (premier_league['Round'] == round + 1)]
        if len(idx_home) == 1:
            idx_home = idx_home.index
            premier_league.loc[idx_home, 'Total Goals So Far Home'] = aux_dict[team]
        else:
            idx_away = premier_league[(premier_league['Away_Team'] == team) & (premier_league['Round'] == round + 1)].index
            premier_league.loc[idx_away, 'Total Goals So Far Away'] = aux_dict[team]

{'Fulham': 0, 'Arsenal': 3, 'Crystal Palace': 1, 'Southampton': 0, 'Liverpool': 4, 'Leeds United': 3, 'West Ham': 0, 'Newcastle': 2, 'West Bromwich Albion': 0, 'Leicester': 3, 'Tottenham Hotspur': 0, 'Everton': 1, 'Sheffield United': 0, 'Wolves': 2, 'Brighton & Hove Albion': 1, 'Chelsea': 3, 'Burnley': 0, 'Man. Utd': 1, 'Man. City': 2, 'Aston Villa': 0}
{'Fulham': 3, 'Arsenal': 5, 'Crystal Palace': 4, 'Southampton': 2, 'Liverpool': 6, 'Leeds United': 7, 'West Ham': 1, 'Newcastle': 2, 'West Bromwich Albion': 2, 'Leicester': 7, 'Tottenham Hotspur': 5, 'Everton': 6, 'Sheffield United': 0, 'Wolves': 3, 'Brighton & Hove Albion': 4, 'Chelsea': 3, 'Burnley': 2, 'Man. Utd': 2, 'Man. City': 5, 'Aston Villa': 1}
{'Fulham': 3, 'Arsenal': 6, 'Crystal Palace': 5, 'Southampton': 3, 'Liverpool': 9, 'Leeds United': 8, 'West Ham': 5, 'Newcastle': 3, 'West Bromwich Albion': 5, 'Leicester': 12, 'Tottenham Hotspur': 6, 'Everton': 8, 'Sheffield United': 0, 'Wolves': 3, 'Brighton & Hove Albion': 6, 'Chelsea

In [89]:
premier_league.tail(40)

,Round,Season,Home_Team,Away_Team,Home Score,Away Score,Total Goals So Far Home,Total Goals So Far Away
79460,22,2021,Tottenham Hotspur,Chelsea,0,1,34,35
79461,23,2021,Aston Villa,Arsenal,1,0,35,27
79462,23,2021,Burnley,Brighton & Hove Albion,1,1,14,24
79463,23,2021,Newcastle,Southampton,3,2,22,27
79464,23,2021,Fulham,West Ham,0,0,18,34
79465,23,2021,Man. Utd,Everton,3,3,46,32
79466,23,2021,Tottenham Hotspur,West Bromwich Albion,2,0,34,18
79467,23,2021,Wolves,Leicester,0,0,23,39
79468,23,2021,Liverpool,Man. City,1,4,43,42
79469,23,2021,Sheffield United,Chelsea,1,2,14,36


In [55]:
aux_dict

{'Fulham': ['0', '0', '1', '2', '2', '1', '0', '0', '1', '0', '0', '0', '1'],
 'Crystal Palace': ['1',
  '1',
  '1',
  '4',
  '0',
  '1',
  '0',
  '1',
  '2',
  '2',
  '1',
  '0',
  '0'],
 'Liverpool': ['4',
  '3',
  '2',
  '2',
  '3',
  '4',
  '2',
  '1',
  '0',
  '0',
  '0',
  '1',
  '0'],
 'West Ham': ['0', '4', '1', '1', '2', '1', '1', '2', '2', '1', '1', '3', '2'],
 'West Bromwich Albion': ['0',
  '3',
  '0',
  '0',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
  '2',
  '1',
  '1'],
 'Tottenham Hotspur': ['0',
  '1',
  '3',
  '2',
  '2',
  '2',
  '0',
  '1',
  '3',
  '1',
  '0',
  '2',
  '4'],
 'Sheffield United': ['0',
  '0',
  '1',
  '0',
  '0',
  '1',
  '2',
  '0',
  '1',
  '1',
  '2',
  '1',
  '0'],
 'Brighton & Hove Albion': ['1',
  '2',
  '1',
  '0',
  '1',
  '1',
  '1',
  '0',
  '3',
  '0',
  '1',
  '0',
  '1'],
 'Burnley': ['0', '0', '0', '0', '1', '1', '2', '1', '1', '3', '0', '1', '0'],
 'Man. City': ['2',
  '2',
  '1',
  '1',
  '5',
  '2',
  '1',
  '2',
  '1',
  '4',
  '1',